In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://about.usps.com/who-we-are/postal-history/pieces-of-mail-since-1789.htm'
sess = requests.Session()
sess.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
res = sess.get(url)


In [3]:
pd.set_option('display.max_rows', 250)
pd.set_option('max_colwidth', 50)

In [4]:
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find_all('table')[0]

In [5]:
dfs = pd.read_html(str(table))
df = dfs[0]
df

,Year,Pieces of Mail Handled,Number of Post Offices,Income,Expenses
0,1789,NaN,75,"$ 7,510","$ 7,560"
1,Income and expenses listed for 1789 are for th...,Income and expenses listed for 1789 are for th...,Income and expenses listed for 1789 are for th...,Income and expenses listed for 1789 are for th...,Income and expenses listed for 1789 are for th...
2,1790,NaN,75,37935,32140
3,1791,NaN,89,46294,36697
4,1792,NaN,195,67443,54530
5,1793,NaN,209,104746,72039
6,1794,NaN,450,128947,89972
7,1795,NaN,453,160620,117893
8,1796,NaN,468,195066,131571
9,1797,NaN,554,213998,150114


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Year                    238 non-null    object
 1   Pieces of Mail Handled  131 non-null    object
 2   Number of Post Offices  238 non-null    object
 3   Income                  238 non-null    object
 4   Expenses                238 non-null    object
dtypes: object(5)
memory usage: 9.4+ KB


In [7]:
# check point
df2 = df.copy()
pattern = r'[0-9]{4}$'
filter = df2['Year'].str.contains(pattern) 
df2 = df2[filter]     # keep only rows with an actual year in the 'Year' col.

In [8]:
df2.head()

,Year,Pieces of Mail Handled,Number of Post Offices,Income,Expenses
0,1789,NaN,75,"$ 7,510","$ 7,560"
2,1790,NaN,75,37935,32140
3,1791,NaN,89,46294,36697
4,1792,NaN,195,67443,54530
5,1793,NaN,209,104746,72039


In [9]:
df2.set_index('Year', inplace=True)
df2.head()

,Pieces of Mail Handled,Number of Post Offices,Income,Expenses
Year,,,,
1789,NaN,75,"$ 7,510","$ 7,560"
1790,NaN,75,37935,32140
1791,NaN,89,46294,36697
1792,NaN,195,67443,54530
1793,NaN,209,104746,72039


In [10]:
df2.columns = ['Pieces_Handled', 'Post_Offices', 'Income', 'Expenses']
df2.head()

,Pieces_Handled,Post_Offices,Income,Expenses
Year,,,,
1789,NaN,75,"$ 7,510","$ 7,560"
1790,NaN,75,37935,32140
1791,NaN,89,46294,36697
1792,NaN,195,67443,54530
1793,NaN,209,104746,72039


In [11]:
df2.drop(['1789'], inplace=True)   # yr 1789 data only accounted for 3 mos.
col_list = ['Pieces_Handled', 'Post_Offices', 'Income', 'Expenses']
df2[col_list] = df2[col_list].apply(pd.to_numeric)

In [12]:
df2['Profit/Loss'] = df2['Income'] - df2['Expenses']
df2.head()

,Pieces_Handled,Post_Offices,Income,Expenses,Profit/Loss
Year,,,,,
1790,NaN,75,37935,32140,5795
1791,NaN,89,46294,36697,9597
1792,NaN,195,67443,54530,12913
1793,NaN,209,104746,72039,32707
1794,NaN,450,128947,89972,38975


In [16]:
# check point
df3 = df2.copy()
df3  =df3.convert_dtypes()

In [17]:
df3.info()


<class 'pandas.core.frame.DataFrame'>
Index: 230 entries, 1790 to 2019
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Pieces_Handled  124 non-null    Int64
 1   Post_Offices    230 non-null    Int64
 2   Income          230 non-null    Int64
 3   Expenses        230 non-null    Int64
 4   Profit/Loss     230 non-null    Int64
dtypes: Int64(5)
memory usage: 11.9+ KB
